In [1]:
import pandas as pd
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer

# Unduh data NLTK
nltk.download('punkt', force=True)
nltk.download('wordnet', force=True)

# Membaca file CSV
data = pd.read_csv('tempat_makan.csv', delimiter=';', encoding='latin1')

# Preprocessing
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Membuat deskripsi tempat makan dalam bentuk list
tempat_makan_deskripsi = data['desk'].tolist()

# Keyword
GREETING_INPUTS = ("halo", "hi", "hai")
GREETING_RESPONSES = ["Halo! Ada yang bisa saya bantu?", "Hi! Butuh rekomendasi tempat makan?", "Halo, selamat datang!"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

def build_vsm_responses(user_response):
    # Inisialisasi respon chatbot
    robo_response = ''

    # Menambahkan input pengguna ke deskripsi tempat makan untuk diproses
    tempat_makan_deskripsi.append(user_response)

    # Membuat representasi vektor TF-IDF
    TfidfVec = TfidfVectorizer(stop_words='english')  # Menggunakan tokenizer default
    tfidf_matrix = TfidfVec.fit_transform(tempat_makan_deskripsi)

    # Menghitung kesamaan kosinus
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix)
    similarity_scores = cosine_similarities.flatten()
    sorted_indices = similarity_scores.argsort()[::-1][1:]

    # Mendapatkan hasil terbaik
    if similarity_scores[sorted_indices[0]] == 0:
        robo_response = "Maaf, saya tidak menemukan tempat makan yang cocok."
    else:
        best_match_index = sorted_indices[0]
        matched_place = data.iloc[best_match_index]
        robo_response = (
            f"Rekomendasi: {matched_place['tempat']}\n"
            f"Alamat: {matched_place['alamat']}\n"
            f"Kategori: {matched_place['kategori']}\n"
            f"Rating: {matched_place['rating']}\n"
            f"Deskripsi: {matched_place['desk']}"
        )

    # Menghapus input pengguna dari list deskripsi untuk menjaga konsistensi data
    tempat_makan_deskripsi.remove(user_response)
    return robo_response



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Testing Chat

In [5]:
# Main Chat Loop
print("MakanBuddy: Hi Saya MakanBuddy. asisten Anda dalam mencari lokasi tempat makanan. Ketik 'selesai' untuk keluar.")
while True:
    user_input = input("Anda: ").lower()
    if user_input == 'selesai':
        print("MakanBuddy: Terima kasih! Selamat makan!")
        break
    elif greeting(user_input):
        print(f"MakanBuddy: {greeting(user_input)}")
    else:
        print(f"MakanBuddy: {build_vsm_responses(user_input)}")

MakanBuddy: Hi Saya MakanBuddy. asisten Anda dalam mencari lokasi tempat makanan. Ketik 'selesai' untuk keluar.
Anda: carikan tempat makan terenak di kota semarang
MakanBuddy: Rekomendasi: Iga Bajog
Alamat: Jalan Panjaitan 84, Semarang Tengah, Semarang
Kategori: BBQ
Rating: 4.5
Deskripsi: Tempat makan iga yang terkenal di Semarang.
Anda: dimana gudeg legendaris di Yogyakarta
MakanBuddy: Rekomendasi: Gudeg Yu Djum
Alamat: Jl. Wijilan No. 167, Yogyakarta
Kategori: Gudeg
Rating: 4.7
Deskripsi: Tempat legendaris untuk menikmati gudeg khas Yogyakarta
Anda: lokasi sate legendaris khas bandung
MakanBuddy: Rekomendasi: Sate Maulana Yusuf
Alamat: Jl. Maulana Yusuf No. 21, Bandung
Kategori: Sate
Rating: 4.6
Deskripsi: Sate legendaris bandung dengan daging empuk dan bumbu kacang lezat.
Anda: temukan restoran yang menyediakan steak di surabaya
MakanBuddy: Rekomendasi: Boncafé
Alamat: Jl. Manyar Kertoarjo No. 41, Surabaya
Kategori: Steak
Rating: 4.6
Deskripsi: Restoran steak di surabaya dengan suas